In [1]:
# TMDBML : TMDB 머신러닝 모델 만들기
import pandas as pd
import numpy as np

<pre>
# vote_average를 가중치를 부여
# vote = (v/(v+m))*R + (m/(v+m))*C
# v : 개별 영화에 투표한 횟수, m : 평점을 부여하기 위한 최소 투표 횟수
# R : 개별 영화에 대한 평균 평점(=vote_average), C : 전체 영화에 대한 평균 평점
</pre>

In [2]:
df = pd.read_csv("data/movie_data_final.csv")
df.head(1)  # title : str

,title,poster_path,release_date,budget,genres,country,main_charactor,director,runtime,overview,vote_average,vote_count
0,{616037: '토르: 러브 앤 썬더'},https://image.tmdb.org/t/p/original/bZLrpWM065...,2022-07-06,250000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},119,"이너피스를 위해 자아 찾기 여정을 떠난 천둥의 신 토르. 그러나, 우주의 모든 신들...",6.778,1802


In [3]:
# m : 투표 횟수의 평균 * 60%
# C : 평균 평점 * 60 %
m = df.vote_average.mean() * 60 / 100
C = df.vote_count.mean() * 60 / 100
print("m : ", m)
print("C : ", C)

m :  3.9188531981279198
C :  1491.941185647426


In [4]:
v = df.vote_count
R = df.vote_average
df['result'] = (v/(v+m))*R + (m/(v+m))*C
df.head(1)

,title,poster_path,release_date,budget,genres,country,main_charactor,director,runtime,overview,vote_average,vote_count,result
0,{616037: '토르: 러브 앤 썬더'},https://image.tmdb.org/t/p/original/bZLrpWM065...,2022-07-06,250000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},119,"이너피스를 위해 자아 찾기 여정을 떠난 천둥의 신 토르. 그러나, 우주의 모든 신들...",6.778,1802,10.000812


In [5]:
import ast #형태가 리스트/딕셔너리인 문자열을 다시 리스트/딕셔너리 형태로 바꿀 수 있다.

In [6]:
# title: 이름, id/ genres: to_categorical/ country: to_categorical/ charactor: 이름, id/ director: 이름, id/ result 0 or 1
# numpy 배열 (5302,)
# df['genres'] = df['genres'].apply(literal_eval) 로 다시 짤 수 있다.
df_list = []
title, budget, genres, country, charactor, director, result = [], [], [], [], [], [], []
for idx in df.index:
    title.append([ast.literal_eval(df.title[idx])])
    budget.append([int(df.budget[idx])])
    genres.append(ast.literal_eval(df.genres[idx]))
    country.append(ast.literal_eval(df.country[idx]))
    charactor.append([ast.literal_eval(df.main_charactor[idx])])
    director.append([ast.literal_eval(df.director[idx])])
    result.append([1 if df.vote_average[idx] >= 0.5 else 0])
df_list.append([title, budget, genres, country, charactor, director, result])
df_arr = np.array(df_list)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35540\2465986390.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_arr = np.array(df_list)


In [7]:
# for title, charactor, director
class SeperateID_Name:
    def __init__(self, column):
        self.id = []
        self.name = []
        self.column = column

    def print(self):
        for col in self.column:
            for key, value in col[0].items():
                self.id.append(int(key))
                self.name.append(str(value))
        return self.id, self.name

In [8]:
# title : dictionary about title_ID & title_Name
sep_title = SeperateID_Name(title)
title_ID, title_Name = sep_title.print()
# charactor : dictionary about char_ID & char_Name
sep_charactor = SeperateID_Name(charactor)
char_ID, char_Name = sep_charactor.print()
# director : dictionary about dir_ID & dir_Name
sep_director = SeperateID_Name(director)
dir_ID, dir_Name = sep_director.print()

In [9]:
# country dictionry name to id 만들기
country_data = []
for cs in country:
    for c in cs:
        country_data.append(c['name'])
country_dict = {}
i = 0
for l in country_data:
    try:
        country_dict[l]  # 찾으면 패스 못찾으면 선언 및 아이디 부여
    except:
        country_dict[l] = i
        i += 1

In [10]:
# for genres, country
class SeperateID_Name_list:
    def __init__(self, column):
        self.ids, self.id = [], []
        self.names, self.name = [], []
        self.column = column

    def print(self):
        for cols in self.column:
            for col in cols:
                self.id.append(list(col.values())[0])
                self.name.append(list(col.values())[1])
            self.ids.append(self.id)
            self.names.append(self.name)
            self.id = []
            self.name = []
        return self.ids, self.names

In [11]:
# genres
sep_genres = SeperateID_Name_list(genres)
genre_ID, genre_Name = sep_genres.print()
# country
sep_country = SeperateID_Name_list(country)
country_ID, country_Name = sep_country.print()

In [12]:
# country_ID id 저장
country_I = []
country_ID = []
for names in country_Name:
    for name in names:
        country_I.append(country_dict[name])
    country_ID.append(country_I)
    country_I = []

- Id로 이루어진 데이터프레임(머신러닝용1)

In [13]:
df.head(1)

,title,poster_path,release_date,budget,genres,country,main_charactor,director,runtime,overview,vote_average,vote_count,result
0,{616037: '토르: 러브 앤 썬더'},https://image.tmdb.org/t/p/original/bZLrpWM065...,2022-07-06,250000000,"[{'id': 28, 'name': '액션'}, {'id': 12, 'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",{3894: 'Christian Bale'},{55934: 'Taika Waititi'},119,"이너피스를 위해 자아 찾기 여정을 떠난 천둥의 신 토르. 그러나, 우주의 모든 신들...",6.778,1802,10.000812


In [14]:
# title	budget	genres	country	main_charactor	director	vote_average	vote_count	result
df_ID = pd.DataFrame({'title': title_ID, 'budget':df.budget, 'genres':genre_ID, 'country':country_ID, 'charactor': char_ID, 'director':dir_ID, 'vote_score':df.vote_average, 'vote_count':df.vote_count,
'vote_weight':df.result})

In [15]:
df_ID.head(1)

,title,budget,genres,country,charactor,director,vote_score,vote_count,vote_weight
0,616037,250000000,"[28, 12, 14]",[0],3894,55934,6.778,1802,10.000812


In [16]:
df_ID.to_csv('data/movie_data_ID.csv', index=False)

- Name으로 이루어진 데이터프레임(분석용)

In [17]:
# title	budget	genres	country	main_charactor	director	vote_average	vote_count	result
df_Name = pd.DataFrame({'title': title_Name, 'budget':df.budget, 'genres':genre_Name, 'country':country_Name, 'charactor': char_Name, 'director':dir_Name, 'vote_score':df.vote_average, 'vote_count':df.vote_count,
'vote_weight':df.result})

In [18]:
df_Name.head(1)

,title,budget,genres,country,charactor,director,vote_score,vote_count,vote_weight
0,토르: 러브 앤 썬더,250000000,"[액션, 모험, 판타지]",[United States of America],Christian Bale,Taika Waititi,6.778,1802,10.000812


In [19]:
df_Name.to_csv('data/movie_data_Name.csv', index=False)

- Id로 이루어진 데이터프레임(머신러닝용2)

- genres, country columns

In [20]:
genres_list = ["Action", "Adventure" ,"Fantasy", "Family", "Animation", "Comedy", "Science Fiction", "Drama", "Horror", "Thriller", "Crime", "Mystery", "Music", "History", "TV Movie", "War", "Romance", "Documentary", "Western"]

In [21]:
# genres 종류에 따라 나온 결과 값을 데이터프레임 변형, 해당하는 값만 1로 부여하고 나머지는 0으로 부여
# 가중치 값은 반올림하여 정수형으로 바꿈.
df_genre_each = pd.DataFrame(columns=genres_list)
for idx in df.index:
    genres = df_Name.genres[idx]
    # 한 줄 0으로 초기화
    df_genre_each.loc[idx] = 0
    for genre in genres:
        df_genre_each.loc[idx][str(genre)] = 1  # 한 줄 값 부여

In [22]:
df_genre_each.to_csv("data/movie_genres_data.csv", index=False)